<a href="https://colab.research.google.com/github/BrownParticleAstro/dmtools/blob/podman-python-dash/notebooks/DMTOOLSDataParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
!pip install --upgrade certifi

In [2]:
import ssl
import urllib
context = ssl._create_unverified_context()
from urllib.request import Request, urlopen
import json
import pandas as pd

In [3]:
from itertools import cycle

from decimal import *

## Decimal was to maintain accuracy of cross section data

In [4]:
def setup_decimal_precision():
    getcontext().prec = 24
    print(Decimal(748327402479023).sqrt())
setup_decimal_precision()

27355573.5176402945763287


# Set Up Request Header

In [5]:
#request_header = {'dmtool-userid': '1' ,'dmtool-apikey': 'CC0EF1DD_C9AE_11EE_BCAD_0EC372396BDC'} # dev server
#request_header = {'dmtool-userid': '1' ,'dmtool-apikey': '81D34E22_CC0F_11EE_8826_FE852FEDCE00'} ## prod server
# 58E88A3E_37DC_11EF_BCE0_FADBD26D994C

def create_request_header(dmtool_userid_in,dmtool_apikey_in ):
    request_header_out = {'dmtool-userid': dmtool_userid_in ,'dmtool-apikey': dmtool_apikey_in}
    return request_header_out
request_header = create_request_header('1','58E88A3E_37DC_11EF_BCE0_FADBD26D994C' )

# Set Up Request URL

In [6]:
api_server = "https://dmtools.het.brown.edu/"
data_api = "dmtool/fastapi_data/open/data/"
fastapi_url_data = api_server + data_api + "read_a_data/" + "?id_in=6"
fastapi_url_data

'https://dmtools.het.brown.edu/dmtool/fastapi_data/open/data/read_a_data/?id_in=6'

In [7]:
request = Request(fastapi_url_data, headers=request_header)
r = urllib.request.urlopen(request, context=context)

#r = urllib.request.urlopen(fastapi_url_plots,context=context, headers = request_header)
#print(r.read())
string = r.read().decode('utf-8')
data_json_obj = json.loads(string)
#all_plots_json_obj[1]
data_df = pd.DataFrame(data_json_obj)
data_df['row_id'] = data_df.index

In [8]:
fastapi_url_all_data = api_server + data_api + "read_all_data"
request = Request(fastapi_url_all_data, headers=request_header)
r = urllib.request.urlopen(request, context=context)

#r = urllib.request.urlopen(fastapi_url_plots,context=context, headers = request_header)
#print(r.read())
string = r.read().decode('utf-8')
data_all_json_obj = json.loads(string)
#all_plots_json_obj[1]
data_all_df = pd.DataFrame(data_all_json_obj)
data_all_df['row_id'] = data_all_df.index

In [9]:
data_all_df[['id','data_label','data_values']].head(5)

,id,data_label,data_values
0,NaN,NaN,NaN
1,1.0,"Baer et. al., 2003, constraints on mSUGRA, SI",{[60 4e-43;70 4e-43;160 5.5e-44;320 2.5e-44;63...
2,5.0,"Baltz et. al., 2003, SUSY, 0.05< Omega, h^2 < ...",{[50 5e-43;75 7e-43;80 4e-42;150 1e-41;150 3e-...
3,6.0,"Baltz et. al., 2004, mSUGRA without Delta-a_mu...",{[46.60526838287 6.751984523073e-51;45.4804689...
4,7.0,"Baltz et. al., 2004, mSUGRA with Delta-a_mu wi...",{[57.47842548777 5.743773748827e-47;55.4102033...


In [10]:
data_df[['id', 'subject', 'data_values', 'data_label']]
list_of_id = data_all_df['id'].tolist()
#list_of_id

In [12]:
#data_4_parsing

In [25]:
def parse_data(all_data_in, data_2_parse):
    row_id = 0
    for index, row in data_2_parse.iterrows():
        id = row['id']
        #print('id from data >>>>', id)
        data_id = row['id']
        data_label = row['data_label']
        data_string = row['data_values']

        data_string = data_string.replace("{[", "")
        data_string = data_string.replace("]}", "")

        data_series = data_string.split("]")

        for l in range(0,len(data_series)):
            series_id = 0
            trace_id = l + 1
            try:
                trace_name = str(l + 1) + "_" + row['data_label']
            except:
                trace_name = str(l + 1) + "_" + 'trace_name'
            data_label = row['data_label']
            subject = 'data_data'
            single_set = data_series[l]
            set_list = single_set.split(";")
            for i in set_list:
                ## the following was added due to a different approach to data_string format
                r0 = i.replace(',[','')
                r1 = r0.replace('  ',' ')
                r2 = r1.replace('  ',' ')
                r3 = r2.replace('  ',' ')
                r4 = r3.replace('\r\n','')
                r5 = r4.replace('\t',' ')
                r6 = r5.replace(',',' ')
                r7 = r6.replace('  ',' ')
                r8 = r7.replace('  ',' ')
                s = r8.lstrip()
                z = s.split(" ");
                try:
                    raw_y = z[1]
                    raw_x = z[0].replace(",[", "")
                except:
                    raw_y = '0'
                    raw_x = '0'

                try:
                    masses =  Decimal(raw_x)
                    masses_as_float = float(raw_x)
                    #cross_sections = float(raw_y)/scale_factor
                    raw_y_l = raw_y.split('e')
                    raw_y_n = raw_y_l[0]
                    try:
                        raw_y_e = raw_y_l[1]
                    except:
                        raw_y_e = 0
                    cross_sections_as_float = float(raw_y)
                    cross_sections = Decimal(raw_y)
                    cross_sections_n = Decimal(raw_y_n)
                    cross_sections_e = Decimal(raw_y_e)
                    if cross_sections == 0:
                        print("this raw_y gave 0 >>>>" , raw_y)
                except:
                    masses = 0
                    cross_sections = 0
                try:
                    #appendthis = [subject,data_id,row_id,series_id,trace_id,trace_name,data_label,raw_x,raw_y,masses,cross_sections,cross_sections_n,cross_sections_e, cross_sections_as_float]
                    appendthis = [subject,data_id,row_id,series_id,trace_id,raw_x,raw_y,masses,cross_sections,cross_sections_n,cross_sections_e, cross_sections_as_float]
                except:
                    appendthis = [subject,data_id,row_id,series_id,trace_id,'0','0','0','0','0','0','0']

                all_data_in.append(appendthis)
                row_id += 1
                series_id += 1

        #keys = ['subject','data_id','row_id','limit_id','series_id','trace_id','trace_name','data_label','raw_x','raw_y','masses','cross_sections','cross_sections_n','cross_sections_e']
        #data_data_lod = [dict(zip(keys, l)) for l in data_data ]
    return all_data_in

In [26]:
all_data = []
for i in list_of_id:
    #print(i)
    data_4_parsing = data_all_df[data_all_df['id']==i]
    all_data = parse_data(all_data, data_4_parsing)
    #print(data_4_parsing)

this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0
this raw_y gave 0 >>>> 0


In [29]:
all_data_df = pd.DataFrame(data=all_data, columns=['subject','data_id','row_id','series_id','trace_id','raw_x','raw_y','masses','cross_sections','cross_sections_n','cross_sections_e', 'cross_sections_as_float'])

In [30]:
all_data_df

,subject,data_id,row_id,series_id,trace_id,raw_x,raw_y,masses,cross_sections,cross_sections_n,cross_sections_e,cross_sections_as_float
0,data_data,1.0,0,0,1,60,4e-43,60,4E-43,4,-43,0.0
1,data_data,1.0,1,1,1,70,4e-43,70,4E-43,4,-43,0.0
2,data_data,1.0,2,2,1,160,5.5e-44,160,5.5E-44,5.5,-44,0.0
3,data_data,1.0,3,3,1,320,2.5e-44,320,2.5E-44,2.5,-44,0.0
4,data_data,1.0,4,4,1,630,3.2e-44,630,3.2E-44,3.2,-44,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
46476,data_data,872.0,4641,20,43,886.472596912,9.77113097016e-10,886.472596912,9.77113097016E-10,9.77113097016,-10,0.0
46477,data_data,872.0,4642,21,43,887.772513547,9.45046878351e-10,887.772513547,9.45046878351E-10,9.45046878351,-10,0.0
46478,data_data,872.0,4643,22,43,889.912092833,9.28362138213e-10,889.912092833,9.28362138213E-10,9.28362138213,-10,0.0
46479,data_data,872.0,4644,23,43,892.397652621,9.10355763309e-10,892.397652621,9.10355763309E-10,9.10355763309,-10,0.0


In [ ]:
#print(fastapi_url_plots)